In [ ]:
import requests, pandas as pd, numpy as np
import csv
import time
import urllib.parse

def read_secrets(file_path):
    with open(file_path) as f:
        return {
            line.split('=', 1)[0].strip(): line.split('=', 1)[1].strip()
            for line in f
            if '=' in line and not line.strip().startswith('#') and line.strip()
        }

# Example usage
secrets = read_secrets('.env')
secrets


: 

In [ ]:
import requests
import csv
import time
import urllib.parse

# --- Your secrets ---
RIOT_API_KEY = "<insert here>"
gameName = "<insert here>"
tagLine = "<insert here>"

# --- Headers ---
HEADERS = {"X-Riot-Token": RIOT_API_KEY}

# --- CSV Setup ---
csv_file = open("tft_match_data.csv", mode="w", newline="", encoding="utf-8")
csv_writer = csv.DictWriter(csv_file, fieldnames=["puuid", "gameName", "tagLine", "summonerLevel", "match_id"])
csv_writer.writeheader()

# --- Helper Functions ---
def get_summoner_by_name(name):
    encoded_name = urllib.parse.quote(name)
    url = f"https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/{encoded_name}"
    r = requests.get(url, headers=HEADERS)
    if r.status_code == 200:
        return r.json()
    else:
        print(f"[Error] get_summoner_by_name: {r.status_code} {r.text}")
        return None

def get_account_by_puuid(puuid):
    url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}"
    r = requests.get(url, headers=HEADERS)
    if r.status_code == 200:
        return r.json()
    else:
        print(f"[Error] get_account_by_puuid: {r.status_code} {r.text}")
        return None

def get_match_ids_by_puuid(puuid, count=5):
    url = f"https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count={count}"
    r = requests.get(url, headers=HEADERS)
    if r.status_code == 200:
        return r.json()
    else:
        print(f"[Error] get_match_ids_by_puuid: {r.status_code} {r.text}")
        return []

def get_match_details(match_id):
    url = f"https://americas.api.riotgames.com/tft/match/v1/matches/{match_id}"
    r = requests.get(url, headers=HEADERS)
    if r.status_code == 200:
        return r.json()
    else:
        print(f"[Error] get_match_details: {r.status_code} {r.text}")
        return None

# --- Start Process ---
seen_puuids = set()
queue = []

# Step 1: Get initial summoner
summoner = get_summoner_by_name(gameName)
if not summoner:
    exit("Could not retrieve initial summoner.")
puuid = summoner["puuid"]
seen_puuids.add(puuid)
queue.append(puuid)

# Breadth-first loop through PUUIDs
while queue:
    current_puuid = queue.pop(0)
    match_ids = get_match_ids_by_puuid(current_puuid, count=5)

    for match_id in match_ids:
        match_data = get_match_details(match_id)
        if not match_data:
            continue
        participants = match_data.get("metadata", {}).get("participants", [])
        
        for p_puuid in participants:
            if p_puuid not in seen_puuids:
                seen_puuids.add(p_puuid)
                queue.append(p_puuid)

            account_info = get_account_by_puuid(p_puuid)
            if not account_info:
                continue

            summoner_info = get_summoner_by_name(account_info["gameName"])
            if not summoner_info:
                continue

            # Save data to CSV
            csv_writer.writerow({
                "puuid": p_puuid,
                "gameName": account_info["gameName"],
                "tagLine": account_info["tagLine"],
                "summonerLevel": summoner_info["summonerLevel"],
                "match_id": match_id
            })

        time.sleep(1.2)  # rate limiting

csv_file.close()
print("✅ Done. Data saved to 'tft_match_data.csv'")
